# GW Releases Presentation

In [1]:
%%capture
import sys
!{sys.executable} -m pip install google-auth-oauthlib
!{sys.executable} -m pip install google-api-python-client
!{sys.executable} -m pip install xhtml2pdf
!{sys.executable} -m pip install pdfrw

In [2]:
import requests
import pandas as pd
import re
from modules.Presentations import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [3]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

We get the json file with the information from https://wmwaredata.s3.us-east-2.amazonaws.com/gw_releases.json

In [4]:
url = 'https://wmwaredata.s3.us-east-2.amazonaws.com/gw_releases.json'
r = requests.get(url, allow_redirects=True)
open('data/gw_releases.json', 'wb').write(r.content)

8144

In [5]:
df = pd.read_json('data/gw_releases.json')
df = df.rename(columns={'sub_repo_commit_url': 'sub_repo_url'})

repos = []
dates = []
tags = []
hashes = []
descriptions = []

for i in range(len(df)):
    # Repo
    repo = df['repo_name'].iloc[i] + '\n\n' + df['repo_url'].iloc[i]
    repos.append(repo)
    # Date
    d = df['release_date'].iloc[i]        
    if d is not None:
        d = d.split('T')
        date = d[0] + '\n\n' + d[1][:-1]
    else:
        date = ''
    dates.append(date)
    # Version / Tag
    t = df['version'].iloc[i] 
    tags.append(t)
    # Hash 
    h = df['hash'].iloc[i]
    hashes.append(h)
    # Notes / Description
    content = re.sub('<.*?>', '', df['release_notes'].iloc[i])
    des = wrap_by_word(content, n=20)
    descriptions.append(des)
    
    # Sub Repo
    s_name = df['sub_repo_name'].iloc[i]
    if s_name is not None:
        s_repo = s_name + '\n\n' + df['sub_repo_url'].iloc[i]
        repos.append(s_repo)
        # date
        dates.append(date)
        # tag
        tags.append(t)
        # Sub Hash
        s_h = df['sub_hash'].iloc[i]
        if s_h is not None:
            hashes.append(s_h)
        # notes
        descriptions.append(des)

df = pd.DataFrame()
df['Repo'] = repos
df['Date'] = dates
df['Version'] = tags
df['Hash'] = hashes
df['Notes'] = descriptions


# Sort columns
df = df[['Repo', 'Date', 'Version', 'Hash', 'Notes']]

# drop repeated rows
df = df.drop_duplicates().reset_index(drop=True)

df

,Repo,Date,Version,Hash,Notes
0,GW-proxy\n\nhttps://github.com/k8-proxy/GW-proxy,2021-02-22\n\n13:57:35,v0.5.2,d8b3bb3db39c7114f2a58b582746cb51ea653b0b,Merge pull request #359 from k8-proxy/svf97-pa...
1,aws-jmeter-test-engine\n\nhttps://github.com/k...,2021-02-19\n\n11:39:43,v0.3,07e073d2c7781d2073ef42199903507d7c60ece2,Update README.md (#415) - Simplified instructi...
2,gp-load-balancer\n\nhttps://github.com/k8-prox...,2021-02-15\n\n11:04:31,v0.1.1,82c3df98dc137c3b7095d3041d9ba57c3b8a2c6d,Merge pull request #32 from naderaly/main fix ...
3,icap-client\n\nhttps://github.com/k8-proxy/ica...,2021-02-17\n\n19:33:50,v0.2.1,a112f474f1b3ebbddf15700c784dd28bcbefcf4e,Update go.mod\n
4,icap-infrastructure\n\nhttps://github.com/k8-p...,2021-02-15\n\n12:18:08,v1.0.1,9027b550a263af153220a3a98377b8e34cf4e7de,update deployment workflow (#55)\n
5,k8-rebuild\n\nhttps://github.com/k8-proxy/k8-r...,2021-02-22\n\n14:27:56,sow-v0.1,fc542782e003b292cd5951651ba5997003f1ecad,
6,k8-rebuild-file-drop\n\nhttps://github.com/k8-...,2021-02-22\n\n14:27:56,sow-v0.1,79b14699988edbe0fb0c64a3b2c4d4c0eb60587e,
7,k8-rebuild-rest-api\n\nhttps://github.com/k8-p...,2021-02-22\n\n14:27:56,sow-v0.1,34a3f777159ff566102850064d2017a6054b7591,
8,k8-rebuild-folder-to-folder\n\nhttps://github....,2021-02-22\n\n15:30:13,v0.1.2,11be72fb5adf41d53be98fdee083446a7938b622,Merge pull request #40 from k8-proxy/MaNe2020 ...
9,s-k8-proxy-rebuild\n\nhttps://github.com/k8-pr...,2021-01-20\n\n09:34:25,v0.1.1,ffbcabeaba991606b3e86d9f44f08fe4e2f4a26d,Merge pull request #56 from k8-proxy/remove_qu...


In [6]:
create_presentation(df, 
                    file_name='GW Releases Presentation', 
                    table_title='GW Releases', 
                    step = 4, 
                    columns_widths=[100, 60, 60, 200, 280])

The presentation is saved to this folder: https://drive.google.com/drive/u/1/folders/1dELfGV6IMMII97tTjqSXeJwPAMEjtbDX